# DHPCTIndividual

> Hierarchical Perceptual Control Theory individual with Keras model compilation and environment execution

In [ ]:
#| default_exp individual

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gymnasium as gym
import json
import pickle
from typing import Optional, Union, Callable, Dict, List, Set, Any

In [ ]:
#| export
class ElementWiseMultiplyLayer(layers.Layer):
    """Custom layer for element-wise multiplication with learnable weights."""
    
    def __init__(self, units, weight_initializer, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.weight_initializer = weight_initializer
    
    def build(self, input_shape):
        self.output_weights = self.add_weight(
            name='output_weights',
            shape=(self.units,),
            initializer=self.weight_initializer,
            trainable=True
        )
        super().build(input_shape)
    
    def call(self, inputs):
        return inputs * self.output_weights
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
        })
        return config

## DHPCTIndividual Class

The `DHPCTIndividual` class represents a single hierarchical Perceptual Control Theory (PCT) system. It manages:

- **Hierarchy structure**: Levels with configurable units per level
- **Keras model**: Functional API model with perception, reference, comparator, and output layers
- **Environment interaction**: Gymnasium environment execution with fitness evaluation
- **Configuration management**: Save/load complete hierarchy specifications
- **Evolutionary operators**: Mutation, crossover for evolutionary algorithms

In [ ]:
#| export
class DHPCTIndividual:
    """
    Represents a hierarchical Perceptual Control Theory system with Keras model.
    
    This class manages the creation, compilation, execution, and evolution of
    hierarchical control systems following PCT principles.
    """
    
    def __init__(
        self,
        env_name: str,
        levels: List[int],
        activation_funcs: Union[str, List[str]] = "linear",
        weight_types: Union[str, List[str]] = "float",
        fixed_weights: Optional[Set[str]] = None,
        fixed_levels: Optional[Set[int]] = None,
        obs_connection_level: int = 0,
        random_seed: Optional[int] = None
    ):
        """
        Initialize a DHPCTIndividual.
        
        Args:
            env_name: Gymnasium environment identifier (e.g., 'CartPole-v1')
            levels: Number of units per level, bottom to top (e.g., [4, 3, 2])
            activation_funcs: Activation function(s) - single string or list per level
            weight_types: Weight type(s) - 'float', 'boolean', or 'ternary'
            fixed_weights: Set of layer names that should not mutate
            fixed_levels: Set of level indices that should not mutate
            obs_connection_level: Which level receives environment observations (default: 0)
            random_seed: Seed for reproducible weight initialization
            
        Raises:
            ValueError: If levels is empty or configuration is invalid
            EnvironmentError: If environment cannot be created
        """
        # Validation
        if not levels or len(levels) == 0:
            raise ValueError("levels must have at least one element")
        if any(l < 1 for l in levels):
            raise ValueError("All level sizes must be >= 1")
        
        # Store parameters
        self.env_name = env_name
        self.levels = levels
        self.obs_connection_level = obs_connection_level
        self.random_seed = random_seed
        
        # Handle activation functions
        if isinstance(activation_funcs, str):
            self.activation_funcs = [activation_funcs] * len(levels)
        else:
            if len(activation_funcs) != len(levels):
                raise ValueError(f"activation_funcs length ({len(activation_funcs)}) must match levels length ({len(levels)})")
            self.activation_funcs = activation_funcs
        
        # Handle weight types
        if isinstance(weight_types, str):
            self.weight_types = [weight_types] * len(levels)
        else:
            if len(weight_types) != len(levels):
                raise ValueError(f"weight_types length ({len(weight_types)}) must match levels length ({len(levels)})")
            self.weight_types = weight_types
        
        # Validate obs_connection_level
        if obs_connection_level >= len(levels):
            raise ValueError(f"obs_connection_level ({obs_connection_level}) must be < len(levels) ({len(levels)})")
        
        # Initialize state
        self.fixed_weights = fixed_weights if fixed_weights is not None else set()
        self.fixed_levels = fixed_levels if fixed_levels is not None else set()
        self.model = None
        self.weights = {}
        self.fitness = None
        self.history = None
        
        # Validate environment
        try:
            test_env = gym.make(env_name)
            self.env_properties = {
                'observation_space': str(test_env.observation_space),
                'action_space': str(test_env.action_space),
                'observation_shape': test_env.observation_space.shape,
                'action_shape': test_env.action_space.shape if hasattr(test_env.action_space, 'shape') else (test_env.action_space.n,)
            }
            test_env.close()
        except Exception as e:
            raise EnvironmentError(f"Failed to create environment '{env_name}': {e}")
    
    def __repr__(self):
        compiled_status = "compiled" if self.model is not None else "uncompiled"
        fitness_str = f", fitness={self.fitness:.2f}" if self.fitness is not None else ""
        return f"DHPCTIndividual(env='{self.env_name}', levels={self.levels}, {compiled_status}{fitness_str})"
    
    def compile(self):
        """
        Build Keras Functional API model from hierarchy specification.
        
        Creates perception, reference, comparator, and output layers for each level
        following PCT principles. Initializes weights according to weight_types.
        
        Raises:
            RuntimeError: If already compiled
            ValueError: If hierarchy specification is invalid
        """
        if self.model is not None:
            raise RuntimeError("Individual is already compiled")
        
        # Set random seed for reproducibility
        if self.random_seed is not None:
            np.random.seed(self.random_seed)
            tf.random.set_seed(self.random_seed)
        
        # Get environment dimensions
        obs_shape = self.env_properties['observation_shape']
        action_shape = self.env_properties['action_shape']
        obs_dim = obs_shape[0] if len(obs_shape) > 0 else 1
        action_dim = action_shape[0] if len(action_shape) > 0 else 1
        
        # Create input layers
        observations = layers.Input(shape=(obs_dim,), name='Observations')
        
        # Calculate reference input dimension (sum of all level outputs except lowest)
        ref_dim = sum(self.levels[1:]) if len(self.levels) > 1 else self.levels[0]
        references_input = layers.Input(shape=(ref_dim,), name='ReferencesInput')
        
        # Storage for layer outputs
        perception_outputs = []  # One per level
        reference_outputs = []   # One per level
        comparator_outputs = []  # One per level (these become Errors output)
        output_layer_outputs = []  # One per level
        
        # TWO-PASS APPROACH: Build perception/output first, then reference layers
        # This is necessary because reference inputs for lower levels come from
        # output layers of higher levels that don't exist yet in a single pass
        
        # === PASS 1: Build perception, comparator, and output layers ===
        # We'll create temporary references and rebuild them in pass 2
        temp_reference_inputs = []
        
        for level_idx, num_units in enumerate(self.levels):
            level_name = f"{level_idx:02d}"
            
            # === PERCEPTION LAYER ===
            # T014: Create perception layers with correct inputs per level
            if level_idx == self.obs_connection_level:
                # This level connects directly to environment observations
                perception_input = observations
            elif level_idx == 0:
                # Level 0: if not observation level, use observations
                perception_input = observations
            else:
                # Higher levels: perceive output from level below
                perception_input = output_layer_outputs[level_idx - 1]
            
            # Create perception layer (weighted sum with activation)
            perception = layers.Dense(
                units=num_units,
                activation=self.activation_funcs[level_idx],
                use_bias=True,
                name=f'PL{level_name}',
                kernel_initializer=self._get_weight_initializer(self.weight_types[level_idx])
            )(perception_input)
            perception_outputs.append(perception)
            
            # Store temp reference input for this level (will be properly connected in pass 2)
            temp_reference_inputs.append(None)
            reference_outputs.append(None)  # Placeholder
            
            # Create temporary comparator and output for structure
            # These will be rebuilt in pass 2 with correct reference connections
            comparator_outputs.append(None)  # Placeholder
            
            # Create output layer placeholder (just pass through comparator for now)
            # Will be properly built in pass 2
            output_layer_outputs.append(None)  # Placeholder
        
        # === PASS 2: Build reference, comparator, and output layers with correct connections ===
        # Now we build from top to bottom so reference connections work
        for level_idx in reversed(range(len(self.levels))):
            num_units = self.levels[level_idx]
            level_name = f"{level_idx:02d}"
            
            # === REFERENCE LAYER ===
            # T015: Create reference layers with correct inputs per level
            if level_idx == len(self.levels) - 1:
                # Highest level: reference comes from external reference input
                reference_input = references_input
            else:
                # Lower levels: reference comes from output of level above
                # This now works because we're building top-down and higher levels exist
                reference_input = output_layer_outputs[level_idx + 1]
            
            # Create reference layer (weighted sum with activation)
            reference = layers.Dense(
                units=num_units,
                activation=self.activation_funcs[level_idx],
                use_bias=True,
                name=f'RL{level_name}',
                kernel_initializer=self._get_weight_initializer(self.weight_types[level_idx])
            )(reference_input)
            reference_outputs[level_idx] = reference
            
            # === COMPARATOR LAYER ===
            # Subtract perception from reference (error = reference - perception)
            comparator = layers.Subtract(name=f'CL{level_name}')([reference, perception_outputs[level_idx]])
            comparator_outputs[level_idx] = comparator
            
            # === OUTPUT LAYER ===
            # Element-wise multiplication with learnable weights
            # For now, we'll use a simpler approach: just pass through the comparator
            # This will be enhanced in later tasks to support proper weighted outputs
            output_layer_outputs[level_idx] = comparator
        
        # Connect actions to the output of level 0
        # and errors to all comparator outputs
        
        # Actions output: from Level 0 output layer
        if len(self.levels) == 1:
            actions = output_layer_outputs[0]
        else:
            actions = output_layer_outputs[0]
        
        # For discrete action spaces, we need to ensure output matches action dimension
        if action_dim != self.levels[0]:
            # Add a final dense layer to map to action space
            actions = layers.Dense(
                units=action_dim,
                activation='linear',
                use_bias=True,
                name='Actions_final'
            )(actions)
        else:
            actions = layers.Activation('linear', name='Actions')(actions)
        
        # Errors output: concatenate all comparator outputs
        if len(comparator_outputs) > 1:
            errors = layers.Concatenate(name='Errors')(comparator_outputs)
        else:
            errors = layers.Activation('linear', name='Errors')(comparator_outputs[0])
        
        # Build the model
        self.model = keras.Model(
            inputs=[observations, references_input],
            outputs=[actions, errors],
            name='DHPCTIndividual'
        )
        
        # Store initial weights
        self._store_weights()
        
    def _get_weight_initializer(self, weight_type: str):
        """Get Keras weight initializer based on weight type."""
        if weight_type == "float":
            return keras.initializers.GlorotUniform(seed=self.random_seed)
        elif weight_type == "boolean":
            # Boolean: initialize to 0 or 1
            def boolean_init(shape, dtype=None):
                return tf.cast(tf.random.uniform(shape, 0, 2, dtype=tf.int32, seed=self.random_seed), dtype=tf.float32)
            return boolean_init
        elif weight_type == "ternary":
            # Ternary: initialize to -1, 0, or 1
            def ternary_init(shape, dtype=None):
                return tf.cast(tf.random.uniform(shape, -1, 2, dtype=tf.int32, seed=self.random_seed), dtype=tf.float32)
            return ternary_init
        else:
            raise ValueError(f"Unknown weight_type: {weight_type}")
    
    def _create_weight_layer(self, num_units: int, weight_type: str, name: str):
        """Create a constant weight layer for element-wise multiplication."""
        # Create trainable weights as a layer
        initializer = self._get_weight_initializer(weight_type)
        initial_weights = initializer((num_units,))
        
        # Use a Lambda layer to create constant weights (trainable via model)
        # Actually, we should use a simple constant that can be updated
        # For now, create a ones layer that will be weighted
        weight_values = layers.Lambda(
            lambda x: tf.ones_like(x) * initial_weights,
            name=name
        )
        
        # Better approach: use a trainable variable
        # This is tricky in Functional API, so let's use a Dense layer with no bias
        # and input of ones
        return layers.Lambda(lambda x: x, name=name)  # Simplified for now
    
    def _store_weights(self):
        """Store current model weights in weights dictionary."""
        if self.model is None:
            return
        
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                self.weights[layer.name] = [w.copy() for w in layer.get_weights()]
    
    def run(
        self,
        steps: int = 500,
        train: bool = False,
        early_termination: bool = True,
        record_history: bool = False,
        train_every_n_steps: int = 1,
        learning_rate: float = 0.01,
        optimizer: str = "adam",
        error_weight_coefficients: Optional[List[float]] = None,
        render: bool = False
    ) -> float:
        """
        Execute individual in environment and return fitness.
        
        Args:
            steps: Maximum number of environment steps
            train: Enable online learning during execution
            early_termination: Stop when environment returns done=True
            record_history: Record ExecutionHistory
            train_every_n_steps: Frequency of weight updates during training
            learning_rate: Learning rate for online learning
            optimizer: Optimizer name for training ('adam', 'sgd', etc.)
            error_weight_coefficients: Weights for different level errors in training
            render: Render environment during execution
            
        Returns:
            Fitness score (typically cumulative reward)
            
        Raises:
            RuntimeError: If not compiled
            EnvironmentError: If environment interaction fails
        """
        if self.model is None:
            raise RuntimeError("Individual must be compiled before running")
        
        # TODO: Implement environment execution (T021)
        pass
    
    def config(self) -> Dict[str, Any]:
        """
        Return complete configuration dictionary.
        
        Returns:
            Configuration dictionary with all hierarchy parameters and weights
        """
        # TODO: Implement config serialization (T031)
        pass
    
    def save_config(self, filepath: str):
        """
        Save configuration to JSON file.
        
        Args:
            filepath: Path to save configuration file
        """
        # TODO: Implement config saving (T032)
        pass
    
    @classmethod
    def from_config(cls, config: Dict[str, Any]) -> 'DHPCTIndividual':
        """
        Create individual from configuration dictionary.
        
        Args:
            config: HierarchyConfiguration dictionary
            
        Returns:
            New compiled DHPCTIndividual with weights loaded
            
        Raises:
            ValueError: If configuration is invalid
            KeyError: If required keys missing from config
        """
        # TODO: Implement from_config (T034)
        pass
    
    @classmethod
    def load_config(cls, filepath: str) -> 'DHPCTIndividual':
        """
        Load individual from JSON configuration file.
        
        Args:
            filepath: Path to configuration file
            
        Returns:
            New compiled DHPCTIndividual
        """
        # TODO: Implement load_config (T033)
        pass
    
    @classmethod
    def from_legacy_config(cls, legacy_config: Dict[str, Any]) -> 'DHPCTIndividual':
        """
        Create individual from legacy configuration format.
        
        Args:
            legacy_config: Configuration in legacy format
            
        Returns:
            New DHPCTIndividual instance
            
        Raises:
            ValueError: If legacy config cannot be converted
        """
        # TODO: Implement from_legacy_config (T037)
        pass
    
    def to_legacy_config(self) -> Dict[str, Any]:
        """
        Convert to legacy configuration format for backward compatibility.
        
        Returns:
            Configuration dictionary in legacy format
        """
        # TODO: Implement to_legacy_config (T036)
        pass
    
    def mate(self, other: 'DHPCTIndividual') -> tuple['DHPCTIndividual', 'DHPCTIndividual']:
        """
        Perform crossover with another individual to create offspring.
        
        Args:
            other: Another DHPCTIndividual for mating
            
        Returns:
            Tuple of two offspring individuals
        """
        # TODO: Implement mate operation (T071-T074)
        pass
    
    def mutate(
        self,
        weight_prob: float = 0.1,
        struct_prob: float = 0.05
    ) -> 'DHPCTIndividual':
        """
        Mutate weights and optionally structure.
        
        Args:
            weight_prob: Probability of mutating each weight
            struct_prob: Probability of structural mutations (add/remove levels/units)
            
        Returns:
            Self (mutated in place)
        """
        # TODO: Implement mutate operation (T075-T080)
        pass
    
    def evaluate(
        self,
        nevals: int = 1,
        aggregation: str = 'mean'
    ) -> float:
        """
        Evaluate fitness over multiple runs.
        
        Args:
            nevals: Number of evaluation runs
            aggregation: How to aggregate results ('mean', 'max', 'min', 'median')
            
        Returns:
            Aggregated fitness score
        """
        # TODO: Implement evaluate with multiple runs (T081-T082)
        pass

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()